In [1]:
#Bibliotecas 
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import SplineTransformer 

In [2]:
#Dicionário de dados
df=pd.read_csv('../data/processed/data.csv')
df_dict = pd.read_csv('../data/external/dictionary.csv')
display(HTML(df_dict.to_html(index=False)))


variavel,descricao,tipo,subtipo
ID,Identificador único do sujeito,Quantitativo,Discreta
Age,Idade do sujeito,Quantitativo,Discreta
Gender,Gênero do sujeito,Qualitativo,Nominal
Bedtime,Hora em que o sujeito vai para a cama,Cíclica,Cíclica
Wakeup time,Hora em que o sujeito acorda,Cíclica,Cíclica
Sleep duration,Duração total do sono em horas,Cíclica,Cíclica
Sleep efficiency,Representa a porcentagem de tempo que o sujeito passou dormindo enquanto estava na cama,Quantitativo,Contínua
REM sleep percentage,"Porcentagem do tempo total de sono que foi gasto no estágio REM, fase do sono associada aos sonhos",Quantitativo,Contínua
Deep sleep percentage,"Porcentagem do tempo total de sono que foi gasto no estágio de sono profundo, fase do sono que é mais restauradora",Quantitativo,Contínua
Light sleep percentage,"Porcentagem do tempo total de sono que foi gasto no estágio de sono leve, fase do sono mais superficial",Quantitativo,Contínua


# 1. Variável-alvo e Variáveis preditoras

In [3]:
#Variável-alvo 
target_variable = "Sleep efficiency"

#Variável Nominais
nominal_variables=(
    df_dict
    .query("subtipo == 'Nominal' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável Cíclica
#cyclic_variables=(
 #   df_dict
  #  .query("subtipo == 'Cíclica' and variavel != @target_variable ")
   # .variavel
    #.to_list()
#)



#Variável Contínuas
continuous_variables=(
    df_dict
    .query("subtipo == 'Contínua' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável discreta
discrete_variables=(
    df_dict
    .query("subtipo == 'Discreta' and variavel != @target_variable ")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_variable],axis=1)
y = df[target_variable]


# 2. Pipeline de Pré-processamento de dados

In [4]:
#Tratamento de dados faltantes e discrepantes, codificação e normalização

#Pipeline para variáveis nominais 
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False)),
])

#Pipeline para variáveis contínuas
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')),
    ("normalization", StandardScaler()),
])

#Pipeline para variáveis discreta
discrete_preprocessor= Pipeline(steps=[
    ("missing", KNNImputer()),
    ("normalization", StandardScaler()),
])

#Pipeline para variáveis cíclicas
#cyclic_preprocessor = Pipeline(steps=[
  #  ("missing", SimpleImputer(strategy="mean")),  
   # ("cyclical_encoding", SplineTransformer()), 
#])


In [5]:
def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )


X["Bedtime"] = X["Bedtime"].astype("datetime64[ns]")
X["Wakeup time"] = X["Wakeup time"].astype("datetime64[ns]")
cyclical_columns = []
granularidades = {
    "year": None,
    "month": 12,
    "day": 31,
    "hour": 24,
    "minute": 60
}
transformers = []
for coluna in ["Bedtime", "Wakeup time"]:
    for granularidade in granularidades.keys():
        new_column_name = f"{coluna}_{granularidade}"
        X[new_column_name] = getattr(X[coluna].dt, granularidade)
        if granularidade != "year":
            cyclical_columns.append(new_column_name)
            transformers.append((
                new_column_name,
                periodic_spline_transformer(period=granularidades[granularidade]),
                [new_column_name]
            ))
    X = X.drop(columns=[coluna])

In [10]:
## Criação do ColumnTransformer para processar cada tipo de variável
preprocessor = ColumnTransformer(
    transformers + [
        ("nominal", nominal_preprocessor, nominal_variables),
        ("continuous", continuous_preprocessor, continuous_variables),
        ("discrete", discrete_preprocessor, discrete_variables),
    ]
)
#preprocessor= ColumnTransformer([
    #("nominal", nominal_preprocessor, nominal_variables),
    #("continuous", continuous_preprocessor, continuous_variables),
    #("discrete", discrete_preprocessor, discrete_variables),
    #("cyclic", cyclic_preprocessor,cyclic_variables ),
#])

# 3. Seleção de modelos

Nesta etapa de modelagem preditiva serão selecionados quatro modelos de aprendizado de máquina que serão testados utilizando um método de validação.

## 3.1. Definição dos modelos e métricas

#### Modelos:
Serão utilizados os seguintes modelos de regressão:

1. **Linear Regression (Regressão Linear):** Um modelo fundamental de regressão que ajusta uma linha reta aos dados para prever uma variável contínua. É simples, rápido e fácil de interpretar, sendo ideal para relações lineares entre variáveis.

2. **SVR:** Um modelo de regressão que incorpora um termo de regularização para controlar a magnitude dos coeficientes e evitar overfitting. Por ser flexível com a escolha de kernels e parâmetros, pode capturar complexidades não lineares nos dados e é robusto a outliers.

3. **KNN Regressor:** Um modelo de regressão baseado em K-vizinhos mais próximos que prevê valores com base na média dos valores dos vizinhos mais próximos. Simples e eficaz em capturar a estrutura local dos dados, especialmente quando a relação entre variáveis é não linear.

4. **Dummy Regressor:** É um modelo simples de aprendizado de máquina que prevê valores numéricos usando regras básicas, sem realmente aprender com os dados de entrada. Assim como sua contraparte de classificação , ele serve como uma linha de base para comparar o desempenho de modelos de regressão mais complexos.

Cada um destes modelos serão testados com diferentes conjuntos de hiper-parâmetros a fim de encontrar o modelo e configuração mais eficazes para o conjunto de dados.

#### Métricas:

1. **neg_mean_absolute_error (MAE)** Média dos erros absolutos das previsões. Valores menores indicam melhor desempenho.
   
2. **neg_mean_squared_error (MSE):** Média dos quadrados dos erros. Penaliza erros maiores mais severamente do que o MAE.

   
3. **neg_mean_absolute_percentage_error (MAPE):** Média dos erros absolutos em termos percentuais. Útil para avaliar o impacto relativo dos erros.

4. **r2:**  Coeficiente de determinação. Mede o quão bem o modelo explica a variância dos dados. Um valor próximo de 1 indica bom ajuste.


In [11]:
models = [DummyRegressor(strategy='mean'), LinearRegression(), KNeighborsRegressor(n_neighbors=5), SVR()]
metrics = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]
monte_carlo = ShuffleSplit(n_splits=10, test_size=.2, random_state=42)

In [12]:
results_total = None
for model in models:
    model_name = model.__class__.__name__
    print(f"rodando para o modelo: {model_name}")
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model),
    ])

    scores = cross_validate(
        approach, X, y,
        scoring=metrics, 
        cv=monte_carlo
    )
    results_model = pd.DataFrame(scores)
    results_model['model'] = model_name
    if results_total is None:
        results_total = results_model
    else:
        results_total = pd.concat([results_total, results_model])

rodando para o modelo: DummyRegressor
rodando para o modelo: LinearRegression
rodando para o modelo: KNeighborsRegressor
rodando para o modelo: SVR


In [9]:
results_total.groupby('model').agg(['mean', 'std']).T

model                                         DummyRegressor  \
fit_time                                mean        0.019611   
                                        std         0.005375   
score_time                              mean        0.009966   
                                        std         0.000605   
test_neg_mean_absolute_error            mean       -0.118838   
                                        std         0.006833   
test_neg_mean_squared_error             mean       -0.019283   
                                        std         0.002155   
test_neg_mean_absolute_percentage_error mean       -0.166970   
                                        std         0.015885   
test_r2                                 mean       -0.018441   
                                        std         0.023587   

model                                         KNeighborsRegressor  \
fit_time                                mean             0.021848   
                                        std              0.003117   
score_time                              mean             0.019537   
                                        std              0.013104   
test_neg_mean_absolute_error            mean            -0.046727   
                                        std              0.003580   
test_neg_mean_squared_error             mean            -0.003530   
                                        std              0.000552   
test_neg_mean_absolute_percentage_error mean            -0.062548   
                                        std              0.004871   
test_r2                                 mean             0.809058   
                                        std              0.051258   

model                                         LinearRegression       SVR  
fit_time                                mean          0.041138  0.023677  
                                        std           0.006790  0.001225  
score_time                              mean          0.012965  0.012211  
                                        std           0.003813  0.000573  
test_neg_mean_absolute_error            mean         -0.051972 -0.061173  
                                        std           0.002394  0.002036  
test_neg_mean_squared_error             mean         -0.004300 -0.005234  
                                        std           0.000416  0.000427  
test_neg_mean_absolute_percentage_error mean         -0.069559 -0.080368  
                                        std           0.004194  0.004394  
test_r2                                 mean          0.769957  0.720658  
                                        std           0.040890  0.039671